In [31]:
import pandas as pd
from scipy.stats import nbinom, poisson
import numpy as np
from joblib import Parallel, delayed

def truncNegBin_logCDF(y, n, p):
    f_zero = nbinom.pmf(0, n, p)
    if y > 0:
        return np.log((nbinom.cdf(y, n, p) - nbinom.cdf(0, n, p)) / (1 - f_zero))
    else:
        return np.log(0)

#log.p	logical; if TRUE, probabilities p are given as log(p)
def qnbinom(p, nNbinom, pNbinom, log_p=False):
    # Convert p to array if it's a single value
    if not isinstance(p, (list, np.ndarray)):
        p = np.array([p])
    
    # Set log-probabilities (lower tail)
    n = len(p)
    if log_p:
        logp = p
    else:
        logp = np.log(p)
    
    # Set output and deal with special cases (outputs NA and Inf)
    quantiles = np.full(n, np.nan)
    nna = ~np.isnan(logp)
    nlogp = logp[nna]
    if len(nlogp) == 0:
        return quantiles
    
    quantiles[nna] = np.full(len(nna), np.inf)
    if np.min(nlogp) >= 0:
        return quantiles


    # calculate mean and variance out of n and p
    mean = (nNbinom * (1 - pNbinom)) / pNbinom
    var = (nNbinom * (1 - pNbinom)) / (pNbinom**2)

    # Set log-CDF vector
    lp_max = np.max(nlogp[nlogp < 0])
    upper = int(mean + np.sqrt(var/(1-np.exp(lp_max)))) #Chebychev inequality
    logcdf = nbinom.logcdf(np.arange(upper+1), nNbinom, pNbinom)

    # Compute output
    for i in range(n):
        if nna[i]:
            if logp[i] < 0:
                quantiles[i] = np.sum(logcdf < logp[i])
    
    # Return output
    if len(quantiles) == 1:
        return quantiles[0]
    else:
        return quantiles

#log.p	logical; if TRUE, probabilities p are given as log(p)    
def qnbinom_trunc(p, nNbinom, pNbinom, log_p=False):
    # if f(0)=0 no truncation is needed
    if (1 - nbinom.pmf(0, nNbinom, pNbinom)) == 1:
        print('Juhu Abkürzung')
        return nbinom.ppf(p, nNbinom, pNbinom)
    else:
        # Convert p to array if it's a single value
        if not isinstance(p, (list, np.ndarray)):
            p = np.array([p])
        
        # Set log-probabilities (lower tail)
        n = len(p)
        if log_p:
            logp = p
        else:
            logp = np.log(p)
        
        # Set output and deal with special cases (outputs NA and Inf)
        quantiles = np.full(n, np.nan)
        nna = ~np.isnan(logp)
        nlogp = logp[nna]
        if len(nlogp) == 0:
            return quantiles
        
        quantiles[nna] = np.full(len(nna), np.inf)
        if np.min(nlogp) >= 0:
            return quantiles

        # calculate mean and variance out of n and p
        mean = (nNbinom * (1 - pNbinom)) / pNbinom
        var = (nNbinom * (1 - pNbinom)) / (pNbinom**2)

        # Set log-CDF vector
        lp_max = np.max(nlogp[nlogp < 0])

        # find a adequate upper limit, starting from the extreme conservative chebychev inequality
        upper = int(mean + np.sqrt(var/(1-np.exp(lp_max)))) #Chebychev inequality
        while np.exp(truncNegBin_logCDF(upper-50, nNbinom, pNbinom)) > 0.999:
            upper = upper - 50

        logcdf = np.array([truncNegBin_logCDF(yi, nNbinom, pNbinom) for yi in range(1, int(upper)+1)]) 

        # Compute output
        for i in range(n):
            if nna[i]:
                if logp[i] < 0:
                    quantiles[i] = np.sum(logcdf < logp[i]) + 1 #+1 because 0 is truncated
        
        # Return output
        if len(quantiles) == 1:
            return quantiles[0]
        else:
            return quantiles

In [37]:
def truncNegBin_CDF(y, n, p):
    f_zero = nbinom.pmf(0, n, p)
    if y > 0:
        return (nbinom.cdf(y, n, p) - nbinom.cdf(0, n, p)) / (1 - f_zero)
    else:
        return 0

def truncNegBin_PPF(x, n, p, epsilon=1e-6, max_iterations=100):
    # if f(0)=0 no truncation is needed
    if (1 - nbinom.pmf(0, n, p)) == 1:
        return nbinom.ppf(x, n, p)
    else:
        # Define the range of y where the solution might exist
        lower_bound = 0
        upper_bound = 1000000000  # Adjust this based on the expected range of y

        # Bisection method
        for _ in range(max_iterations):
            y = (lower_bound + upper_bound) / 2
            cdf_value = truncNegBin_CDF(y, n, p)

            if abs(cdf_value - x) < epsilon:
                return np.ceil(y)  # Found a good approximation

            if cdf_value < x:
                lower_bound = y
            else:
                upper_bound = y

        # Return the best approximation if max_iterations is reached
        return np.ceil(y)

def calculate_trunc_nbinom_quantile(quantile, n, p):
    return truncNegBin_PPF(quantile, n, p)

In [38]:
# calculation of quantiles
quantiles = np.arange(0.001, 0.9999, 0.001)
quantiles = [round(q, 3) for q in quantiles] # due to binary inaccuracies

# Example usage
mean = 5883.111111111111
var = 91679334.861111112

n = (mean**2) / (var - mean) # equivalent to r
p = mean / var

a = nbinom.ppf(quantiles, n, p)
b = qnbinom_trunc(quantiles, n, p)
""" trunc_nbinom_quantiles = Parallel(n_jobs=-1)(delayed(calculate_trunc_nbinom_quantile)(quantile, n, p) for quantile in quantiles) #fast way
c = np.array(trunc_nbinom_quantiles) """
print(nbinom.pmf(0, n, p))
a

77569
0.026126063474903656


array([0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
       1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
       2.0000e+00, 2.0000e+00, 2.0000e+00, 2.0000e+00, 2.0000e+00,
       2.0000e+00, 2.0000e+00, 3.0000e+00, 3.0000e+00, 3.0000e+00,
       3.0000e+00, 3.0000e+00, 3.0000e+00, 4.0000e+00, 4.0000e+00,
       4.0000e+00, 4.0000e+00, 5.0000e+00, 5.0000e+00, 5.0000e+00,
       5.0000e+00, 6.0000e+00, 6.0000e+00, 6.0000e+00, 6.0000e+00,
       7.0000e+00, 7.0000e+00, 7.0000e+00, 8.0000e+00, 8.0000e+00,
       8.0000e+00, 9.0000e+00, 9.0000e+00, 9.0000e+00, 1.0000e+01,
       1.0000e+01, 1.1000e+01, 1.1000e+01, 1.1000e+01, 1.2000e

In [39]:
b

array([1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
       1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
       2.0000e+00, 2.0000e+00, 2.0000e+00, 2.0000e+00, 2.0000e+00,
       2.0000e+00, 2.0000e+00, 3.0000e+00, 3.0000e+00, 3.0000e+00,
       3.0000e+00, 3.0000e+00, 4.0000e+00, 4.0000e+00, 4.0000e+00,
       4.0000e+00, 4.0000e+00, 5.0000e+00, 5.0000e+00, 5.0000e+00,
       5.0000e+00, 6.0000e+00, 6.0000e+00, 6.0000e+00, 6.0000e+00,
       7.0000e+00, 7.0000e+00, 7.0000e+00, 8.0000e+00, 8.0000e+00,
       8.0000e+00, 9.0000e+00, 9.0000e+00, 9.0000e+00, 1.0000e+01,
       1.0000e+01, 1.0000e+01, 1.1000e+01, 1.1000e+01, 1.2000e+01,
       1.2000e+01, 1.3000e+01, 1.3000e+01, 1.3000e+01, 1.4000e+01,
       1.4000e+01, 1.5000e+01, 1.5000e+01, 1.6000e+01, 1.6000e+01,
       1.7000e+01, 1.7000e+01, 1.8000e+01, 1.8000e+01, 1.9000e+01,
       1.9000e+01, 2.0000e+01, 2.1000e+01, 2.1000e+01, 2.2000e+01,
       2.2000e+01, 2.3000e+01, 2.4000e+01, 2.4000e+01, 2.5000e

In [99]:
d = b == c

In [100]:
len(d[d == False])

0

In [89]:
mean = 4
var = 6

n = (mean**2) / (var - mean) # equivalent to r
p = mean / var

mean = (n * (1 - p)) / p
var = (n * (1 - p)) / (p**2)
print(mean)
print(var)

4.000000000000001
6.000000000000001
